In [1]:
import json
import numpy as np

import pandas as pd

In [2]:
import os, sys
import traci

SUMO_HOME = os.environ["SUMO_HOME"] #locating the simulator
sys.path.append(SUMO_HOME+"/tools")

sys.path.append("..")
import visualization
import sumolib
from importlib import reload
import traci.constants as tc

In [3]:
indices = {}
moves = {}

In [ ]:
sumo_cmd = ["sumo", "-c", "random_grid/stable.sumocfg" , "--seed", "42", "--no-step-log"]
traci.start(sumo_cmd)

active_vehicles = set()
end_time = traci.simulation.getEndTime()
timestep = 0

while ((end_time > 0) and (timestep < end_time)) or ((end_time<0) and (traci.simulation.getMinExpectedNumber()>0)):
    traci.simulationStep()
    timestep = traci.simulation.getTime()

    #new cars are added:
    new_vehicles = traci.simulation.getDepartedIDList()
    for veh in new_vehicles:
        if veh in indices:
            indices[veh] += 1
        else:
            indices[veh] = 0
        active_vehicles.add(veh)
        try:
            traci.vehicle.subscribe(veh, [tc.VAR_ROAD_ID])
        except:
            pass
    #stop ending (restarting) vehicles:
    starting_vehicles = traci.simulation.getStopEndingVehiclesIDList()
    for veh in starting_vehicles:
        if veh in indices:
            indices[veh] += 1
        else:
            indices[veh] = 0
        active_vehicles.add(veh)
        try:
            traci.vehicle.subscribe(veh, [tc.VAR_ROAD_ID])
        except:
            pass
    #stopping vehicles:
    stopping_vehicles = traci.simulation.getStopStartingVehiclesIDList()
    for veh in stopping_vehicles:
        if veh in active_vehicles:
            active_vehicles.remove(veh)
            try:
                traci.vehicle.unsubscribe(veh)
            except:
                pass
    #arriving vehicles:
    arriving_vehicles = traci.simulation.getArrivedIDList()
    for veh in arriving_vehicles:
        if veh in active_vehicles:
            active_vehicles.remove(veh)

    for veh in active_vehicles:
        edge = traci.vehicle.getSubscriptionResults(veh)[tc.VAR_ROAD_ID]
        if edge[0] != ":":
            print(moves)
            if not ("%s_%d"%(veh, indices[veh]) in moves):
                moves["%s_%d"%(veh, indices[veh])] = []
            moves["%s_%d"%(veh, indices[veh])].append(edge)
            
traci.close()
with open("moves.json", "w") as f:
    json.dump(moves, f)

---

In [6]:
with open("moves.json") as f:
    moves = json.load(f)
    
moves_cleanup = {}
for car_move in moves:
    moves_cleanup[car_move] = [moves[car_move][0]]
    i = 1
    while i<len(moves[car_move]):
        if moves[car_move][i] != moves[car_move][i-1]:
            moves_cleanup[car_move].append(moves[car_move][i])
        i += 1
        
with open("move_cleanedup.json", "w") as f:
    json.dump(moves_cleanup, f)